# Predictor de ventas del area Cosmetica por sectores para Stanhome

*Este proyecto se realiza bajo la solicitud del departamento de Marketing, quienes luego de ver la disminución de ventas del área de Beauty (cosmética), han solicitado la verificación de ventas de sus productos estrellas y la repercusión de las ofertas.*

# Datos disponibles
Se cuenta con varias tablas de Excel, que contienen:
-	Por mes (de los últimos 3 anos), las compras realizadas por área de productos por comprador y catalogo
-	Ofertas realizadas por producto por catalogo
-	Información de los compradores (fechas de alta, fecha de ultima compra, fechas de nacimiento, zona a la que pertenecen, zona de envío de productos)
-	Fechas (semana Stanhome) de catálogos

# Cargamos Librerias

Rutas absoluta

In [ ]:
import sys, os
current_path = os.getcwd()
src_path = os.path.dirname(current_path)
root_path = os.path.dirname(src_path)
def get_root_path(n):
    '''
    Esta función nos permite iterar sobre carpetas para añadir el path de nuestra carpeta raíz
    Argumentos:
        - n (int): el número de veces que iteraremos para llegar a la carpeta deseada
    '''
    path = os.getcwd() # para notebook ||| __file__ --> para .py
    for i in range(n):
        path = os.path.dirname(path)
    print(path)
    sys.path.append(path)

get_root_path(n=2)
sys.path

In [4]:
import pandas as pd
import numpy as np
import sys, os
from src.utils.funciones import * 

# Carga datos
Usamos la data que nos proporciona la empresa

In [5]:
data = pd.read_excel("../data/processed/Data_STH.xlsx")


In [ ]:
data.head(3)

,REGION,ZONA,CICLO,Ciclo_MONTH,SEMANA (SST),YEAR,DAY,#_FACTURA,ID_STANHOME,ITEM,...,PICK,%DTO.,PAGINA,COD-MOV.,AMBITO,PVP.ART.,PVP,Q.FACTURADA,PVP_TOT_FACTURA,PROMO
0,GG,GG,201801,1,2018011,2018,1,3269760,74214,528,...,1,25.0,10,2200,KI,44.9,44.9,1,272.14,desc directo
1,SG,CS,201801,1,2018011,2018,1,3269844,64198,746,...,1,20.0,58,2200,CP,12.0,12.0,1,156.80,desc directo
2,MN,CM,201801,1,2018011,2018,1,3269849,25322,528,...,1,30.0,10,2200,KI,44.9,44.9,1,370.40,desc directo


In [ ]:
data.columns

Index(['REGION', 'ZONA', 'CICLO', 'Ciclo_MONTH', 'SEMANA (SST)', 'YEAR', 'DAY',
       '#_FACTURA', 'ID_STANHOME', 'ITEM', 'COD_Pers_Item', 'DESCR.ITEM',
       'ARTICULO', 'DESCR.ART.', 'PICK', '%DTO.', 'PAGINA', 'COD-MOV.',
       'AMBITO', 'PVP.ART.', 'PVP', 'Q.FACTURADA', 'PVP_TOT_FACTURA', 'PROMO'],
      dtype='object')

# Verificamos datos personales 

Los datos personales que tenemos para identificar al cliente son los ID_STANHOME, y estan anonimizados.  

# Verificamos si tenemos datos nulos

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43038 entries, 0 to 43037
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   REGION           43038 non-null  object 
 1   ZONA             43038 non-null  object 
 2   CICLO            43038 non-null  int64  
 3   Ciclo_MONTH      43038 non-null  int64  
 4   SEMANA (SST)     43038 non-null  int64  
 5   YEAR             43038 non-null  int64  
 6   DAY              43038 non-null  int64  
 7   #_FACTURA        43038 non-null  int64  
 8   ID_STANHOME      43038 non-null  int64  
 9   ITEM             43038 non-null  int64  
 10  COD_Pers_Item    43038 non-null  object 
 11  DESCR.ITEM       43038 non-null  object 
 12  ARTICULO         43038 non-null  int64  
 13  DESCR.ART.       43038 non-null  object 
 14  PICK             43038 non-null  int64  
 15  %DTO.            43038 non-null  float64
 16  PAGINA           43038 non-null  int64  
 17  COD-MOV.    

No tenemos datos nulos. 

# Creamos categorias numericas

In [6]:
data['REGION_ID'] = data['REGION'].apply(mapping_REGION)
data['SEGMENTO_ID'] = data['AMBITO'].apply(mapping_SEGMENTO)
data['ITEM_ID'] = data['DESCR.ART.'].apply(mapping_ITEM)
data['PROMO_ID'] = data['PROMO'].apply(mapping_PROMO)
data['MONTH'] = data['Ciclo_MONTH'].apply(mapping_PROMO)
data['DATE'] = data['YEAR'] + data['MONTH'] + data['DAY']
data['DATE'] = pd.to_datetime(data['DATE'])

# Creamos Dataset por producto con las variables necesarias

In [7]:
df = data[['REGION_ID','CICLO', 'YEAR', 'MONTH', 'DAY',
       'DATE', 'ID_STANHOME', 'ITEM_ID', 'ARTICULO', '%DTO.', 'PAGINA',
       'COD-MOV.', 'SEGMENTO_ID', 'PVP','PVP_TOT_FACTURA', 'PROMO_ID']]
df

,REGION_ID,CICLO,YEAR,MONTH,DAY,DATE,ID_STANHOME,ITEM_ID,ARTICULO,%DTO.,PAGINA,COD-MOV.,SEGMENTO_ID,PVP,PVP_TOT_FACTURA,PROMO_ID
0,2,201801,2018,4,1,1970-01-01 00:00:00.000002023,74214,1,3599,25.0,10,2200,0,44.9,272.14,4
1,3,201801,2018,4,1,1970-01-01 00:00:00.000002023,64198,2,9078,20.0,58,2200,0,12.0,156.80,4
2,4,201801,2018,4,1,1970-01-01 00:00:00.000002023,25322,1,3599,30.0,10,2200,0,44.9,370.40,4
3,1,201801,2018,4,1,1970-01-01 00:00:00.000002023,66123,2,9078,25.0,58,2200,0,7.9,217.68,4
4,3,201801,2018,4,1,1970-01-01 00:00:00.000002023,78909,2,9078,20.0,58,2200,0,7.9,133.10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43033,2,202117,2021,4,1,1970-01-01 00:00:00.000002026,72275,3,9185,20.0,49,2200,1,39.0,155.90,4
43034,4,202117,2021,4,1,1970-01-01 00:00:00.000002026,61465,1,3599,20.0,15,2200,0,39.0,118.85,3
43035,2,202117,2021,4,1,1970-01-01 00:00:00.000002026,21785,1,3599,30.0,15,2200,0,39.0,478.78,3
43036,3,202117,2021,4,1,1970-01-01 00:00:00.000002026,28522,3,9185,18.0,49,2200,1,39.0,80.00,4


In [8]:
df_1 = df[df["ITEM_ID"] == 1]
df_2 = df[df["ITEM_ID"] == 2]


In [ ]:
df_1.head(2)

,REGION_ID,CICLO,YEAR,MONTH,DAY,DATE,ID_STANHOME,ITEM_ID,ARTICULO,%DTO.,PAGINA,COD-MOV.,SEGMENTO_ID,PVP,PVP_TOT_FACTURA,PROMO_ID
0,2,201801,2018,4,1,1970-01-01 00:00:00.000002023,74214,1,3599,25.0,10,2200,0,44.9,272.14,4
2,4,201801,2018,4,1,1970-01-01 00:00:00.000002023,25322,1,3599,30.0,10,2200,0,44.9,370.40,4


In [ ]:
df_2.head(2)

,REGION_ID,CICLO,YEAR,MONTH,DAY,DATE,ID_STANHOME,ITEM_ID,ARTICULO,%DTO.,PAGINA,COD-MOV.,SEGMENTO_ID,PVP,PVP_TOT_FACTURA,PROMO_ID
1,3,201801,2018,4,1,1970-01-01 00:00:00.000002023,64198,2,9078,20.0,58,2200,0,12.0,156.80,4
3,1,201801,2018,4,1,1970-01-01 00:00:00.000002023,66123,2,9078,25.0,58,2200,0,7.9,217.68,4


# Guardamos cada uno de los datasets

In [10]:
df_1.to_excel('../data/processed/df_1.xlsx', sheet_name='df_1')
df_2.to_excel('../data/processed/df_2.xlsx', sheet_name='df_2')

